In [1]:
import csv
import pandas
import statistics

# Remove Duplicates

Remove any duplicates that appear when scraping the data from Youtube. Probably an issue with the scraping script. 

In [2]:
videosdata = pandas.read_csv('channeldata.csv')
print(videosdata.iloc[1]['ThumbnailURL'])

seen = []
result = []
dups =0
for itemnum in videosdata.index:
    if itemnum != 0:
        if videosdata.iloc[itemnum]["ThumbnailURL"] not in seen:
            result.append(videosdata.iloc[itemnum])
            seen.append(videosdata.iloc[itemnum]['ThumbnailURL'])
        else:
            dups +=1

print(f"num of duplicates: {dups} out of {itemnum} items.")

with open("correcteddata.csv", 'a', encoding="utf-8", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['ChannelID', 'ChannelName', 'Title', 'Viewcount', 'ThumbnailURL', 'Time', 'Tags', 'Duration'])
    writer.writerows(result)

KeyError: 'ThumbnailURL'

# Check for data loss after duplicates removed

Double check no data was lost when removing the duplicates.

In [8]:
list2 = pandas.read_csv('channelData.csv')
list1 = pandas.read_csv('correcteddata.csv')
print(list(set(list2).difference(list1)))

[]


# Create new file with Z-Scores

Calculate the zscores for each video. This takes into account only videos by the same creator and only the 7 videos either side (date wise) of the video in question. 

In [9]:
videosdata = pandas.read_csv('correcteddata.csv')
allzscores = []
for videoind in videosdata.index:
    curChannel = str(videosdata.iloc[videoind]['ChannelID'])
    curChanViews = videosdata.iloc[videoind]['Viewcount']
    surrounding = []

    forward = 8
    if videoind >= videosdata.shape[0] - 7 -1:
        forward = videosdata.shape[0]-1 - videoind
        
    for v in range(forward):
        surrounding.append(videosdata.iloc[videoind+v])
    a=7
    for v in range(7):
        surrounding.append(videosdata.iloc[videoind-a])
        a-=1
    
    surrounding = pandas.DataFrame(surrounding).reset_index(drop=True) # resets indexnummbers from original dataframe to up to date indexes
    
    todel = [] 
    for s in surrounding.index: # removes other channels from surrounding 7 videos
        urmum =str(surrounding.iloc[s]['ChannelID'])
        if urmum != curChannel: # if channel not the same
            todel.append(s)
    chansurround = surrounding.drop(index=todel).reset_index(drop=True)

    viewcounts = []
    for i in chansurround.index:
        viewcounts.append(chansurround.iloc[i]['Viewcount'])
    zscore = (curChanViews - statistics.mean(viewcounts)) / statistics.stdev(viewcounts)
    allzscores.append(zscore)

videosdata['Z-Score'] = allzscores
videosdata.to_csv('datazscores.csv')

                     ChannelID    ChannelName  \
0     UCn6SuRuauJlbWsw7eH1FQTQ          ohlai   
1     UCn6SuRuauJlbWsw7eH1FQTQ          ohlai   
2     UCn6SuRuauJlbWsw7eH1FQTQ          ohlai   
3     UCJLZe_NoiG0hT7QCX_9vmqw  I did a thing   
4     UCJLZe_NoiG0hT7QCX_9vmqw  I did a thing   
...                        ...            ...   
3313  UCkjrBN_GAjFJyVvjcI07KkQ    heyxnatalie   
3314  UCkjrBN_GAjFJyVvjcI07KkQ    heyxnatalie   
3315  UCkjrBN_GAjFJyVvjcI07KkQ    heyxnatalie   
3316  UCkjrBN_GAjFJyVvjcI07KkQ    heyxnatalie   
3317  UCkjrBN_GAjFJyVvjcI07KkQ    heyxnatalie   

                                                  Title  Viewcount  \
0     I Made a Michael Reeves Robot That DESTROYS Yo...       2974   
1                       Oli Bailey has just subscribed!       1776   
2     Making the Worlds Largest Umbrella (to escape ...        421   
3                            I put a gun on a robot dog    4814772   
4                   How Many Shirts Will Stop a Bullet?    39